In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.linear_model import LinearRegression

In [42]:
df = pd.read_csv("/content/drive/MyDrive/CSCE464/datasets/464assignment3_ds.csv")
df

,id,age,weight,oxygen,runtime,rstpulse,runpulse,maxpulse
0,1,44,89.47,44.61,11.37,62,178,182
1,2,40,75.07,45.31,10.07,62,185,185
2,3,44,85.84,54.30,8.65,45,156,168
3,4,42,68.15,59.57,8.17,50,166,172
4,5,38,89.02,49.87,9.22,55,178,180
5,6,47,77.45,44.81,11.63,58,176,176
6,7,40,75.98,45.68,11.95,70,176,180
7,8,43,81.19,49.09,10.85,64,162,170
8,9,44,81.42,39.44,13.08,63,174,176
9,10,38,81.87,60.06,8.63,48,170,186


In [43]:
df = df[['age', 'weight', 'runtime', 'rstpulse', 'runpulse', 'maxpulse', 'oxygen']]
df

,age,weight,runtime,rstpulse,runpulse,maxpulse,oxygen
0,44,89.47,11.37,62,178,182,44.61
1,40,75.07,10.07,62,185,185,45.31
2,44,85.84,8.65,45,156,168,54.30
3,42,68.15,8.17,50,166,172,59.57
4,38,89.02,9.22,55,178,180,49.87
5,47,77.45,11.63,58,176,176,44.81
6,40,75.98,11.95,70,176,180,45.68
7,43,81.19,10.85,64,162,170,49.09
8,44,81.42,13.08,63,174,176,39.44
9,38,81.87,8.63,48,170,186,60.06


In [44]:
x_train = df.iloc[:, 0:-1].to_numpy()
y_train = df.iloc[:, [-1]].to_numpy()

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

In [45]:
x_train

array([[-0.52423807,  1.607224  , -0.15796614,  1.04343924,  0.94625375,
         1.02365398],
       [-1.34336004, -0.45383339, -0.22274721,  1.04343924,  1.70239259,
         1.38241122],
       [-0.52423807,  1.08766578, -0.29350807, -1.21912034, -1.43018261,
        -0.65054645],
       [-0.93379906, -1.44428596, -0.31742723, -0.55366164, -0.34998426,
        -0.17220347],
       [-1.75292103,  1.54281595, -0.26510406,  0.11179706,  0.94625375,
         0.78448249],
       [ 0.09010342, -0.1131864 , -0.14500993,  0.51107228,  0.73021408,
         0.30613951],
       [-1.34336004, -0.32358601, -0.12906382,  2.10817316,  0.73021408,
         0.78448249],
       [-0.72901856,  0.422116  , -0.18387857,  1.30962272, -0.7820636 ,
        -0.41137496],
       [-0.52423807,  0.45503567, -0.07275412,  1.17653098,  0.51417441,
         0.30613951],
       [-1.75292103,  0.51944371, -0.2945047 , -0.81984512,  0.08209507,
         1.50199696],
       [-0.52423807, -0.74581651, -0.21975731, -1.

In [46]:
y_train

array([[44.61],
       [45.31],
       [54.3 ],
       [59.57],
       [49.87],
       [44.81],
       [45.68],
       [49.09],
       [39.44],
       [60.06],
       [50.54],
       [37.39],
       [44.75],
       [47.27],
       [51.86],
       [49.16],
       [40.84],
       [46.67],
       [46.77],
       [50.39],
       [39.41],
       [46.08],
       [45.44],
       [54.63],
       [45.12]])

#PyTorch Model

In [47]:
x = torch.FloatTensor(x_train) #Shape of X: 25X6

y = torch.FloatTensor(y_train) #Shape of y: 25X1

w = torch.zeros((6, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([w, b], lr=1e-6)

for epochs in range(1001):
  hypothesis = x.matmul(w) + b #matmul method is for matrix multiplication
  cost = torch.mean((hypothesis - y)**2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epochs % 100 == 0:
    print("Epoch:  {:3d} / {} W1: {:.5f} W2: {:.5f} W3: {:.5f}  W4: {:.5f} W5: {:.5f} W6: {:.5f} b: {:.5f} Hypothesis: {} \
     Cost: {:.5f}".format(epochs, 1000, w[0].item(), w[1].item(), w[2].item(), w[3].item(), w[4].item(), w[5].item(), \
                          b.item(), hypothesis.detach(), cost.item()))

Epoch:    0 / 1000 W1: -0.00000 W2: -0.00000 W3: -0.00000  W4: -0.00000 W5: -0.00001 W6: -0.00000 b: 0.00010 Hypothesis: tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])      Cost: 2292.95630
Epoch:  100 / 1000 W1: -0.00040 W2: -0.00005 W3: -0.00040  W4: -0.00046 W5: -0.00052 W6: -0.00029 b: 0.00961 Hypothesis: tensor([[0.0085],
        [0.0084],
        [0.0112],
        [0.0106],
        [0.0095],
        [0.0088],
        [0.0086],
        [0.0098],
        [0.0088],
        [0.0102],
        [0.0106],
        [0.0079],
        [0.0095],
        [0.0107],
        [0.0099],
        [0.0089],
        [0.0092],
        [0.0102],
        [0.0105],
        [0.0089],
        [0.0062],
       

#Predicted PyTorch Output

In [48]:
x_new = torch.FloatTensor([[44, 89.49, 11.37, 62, 178, 182]])
print("Predicted oxygen absorption rate: ", x_new.matmul(w) + b)

Predicted oxygen absorption rate:  tensor([[-1.8720]], grad_fn=<AddBackward0>)


#TensorFlow Model

In [49]:
x = tf.placeholder(tf.float32, shape=[None, None])
y = tf.placeholder(tf.float32, shape=[None, None])

w = tf.Variable(tf.random_normal([6, 1]))
b = tf.Variable(tf.random_normal([1]))

hypothesis = tf.matmul(x, w) + b

cost = tf.reduce_mean(tf.square(hypothesis - y))

train = tf.train.GradientDescentOptimizer(learning_rate = 1e-5).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(1001):
  _, w_val, b_val, cost_val, hy_val = \
  sess.run([train, w, b, cost, hypothesis],\
           feed_dict={x: x_train, y: y_train})

  if epoch % 100 == 0:
    print(epoch, "w: ", w_val, 'bias: ', b_val, 'cost: ', cost_val, '\n\n hypothesis: ', hy_val)

0 w:  [[ 0.08004989]
 [-0.09080307]
 [ 0.35376972]
 [ 0.2676064 ]
 [-0.9197132 ]
 [-0.7141386 ]] bias:  [1.2866751] cost:  2171.023 

 hypothesis:  [[-0.28010273]
 [-1.1330992 ]
 [ 2.494753  ]
 [ 1.5264558 ]
 [-0.48913658]
 [ 0.49852824]
 [ 0.49430323]
 [ 2.487515  ]
 [ 0.80008066]
 [-0.37360835]
 [ 1.4952006 ]
 [-2.0557747 ]
 [ 0.35034394]
 [ 2.4820747 ]
 [ 1.6556039 ]
 [-1.1993977 ]
 [ 1.7545351 ]
 [ 2.2472472 ]
 [ 2.365726  ]
 [ 2.3555999 ]
 [ 2.6950316 ]
 [ 3.6489596 ]
 [ 2.2499056 ]
 [ 5.003209  ]
 [ 1.0697849 ]]
100 w:  [[ 0.07439779]
 [-0.0904134 ]
 [ 0.3493405 ]
 [ 0.26344892]
 [-0.9218808 ]
 [-0.7139777 ]] bias:  [1.3791351] cost:  2162.4 

 hypothesis:  [[-0.18957615]
 [-1.0400405 ]
 [ 2.5999656 ]
 [ 1.6280718 ]
 [-0.38738096]
 [ 0.5874207 ]
 [ 0.5845822 ]
 [ 2.581261  ]
 [ 0.8900485 ]
 [-0.2662592 ]
 [ 1.5965627 ]
 [-1.9654546 ]
 [ 0.4449237 ]
 [ 2.5804234 ]
 [ 1.7494373 ]
 [-1.1073389 ]
 [ 1.8408611 ]
 [ 2.3405528 ]
 [ 2.463137  ]
 [ 2.4390311 ]
 [ 2.750297  ]
 [ 3.7322168 

#Predicted TensorFlow Output

In [50]:
print("The predicted oxygen absorption rate is ", sess.run(hypothesis, feed_dict={x: [[44, 89.49, 11.37, 62, 178, 182]]}))

The predicted oxygen absorption rate is  [[-284.17087]]


#Sci_Kit Learn Model and Predicted Output

In [51]:
mlr = LinearRegression()
mlr.fit(x_train, y_train)

LinearRegression()

In [52]:
new_score = [[44, 89.49, 11.37, 62, 178, 182]]
print("The predicted oxygen absorption rate is ", mlr.predict(new_score))

y_pred = mlr.predict(x_train)
print("Cost: ", mean_squared_error(y_train, y_pred))

The predicted oxygen absorption rate is  [[-756.20658511]]
Cost:  10.377622719774317
